# State space representation

The "standard" or most commonly used state space representation is 

\begin{align} \dot{x} &= Ax + Bu \\ y &= Cx + Du \end{align}

Take note that Seborg uses a slightly different version:

\begin{align} \dot{x} &= Ax + Bu + Ed\\ y &= Cx \end{align}

This second version can not represent pure gain systems as it effectively assumes $D=0$. It is also possible to stack $u$ and $d$ from the bottom form into one input vector, so the $E$ matrix really doesn't add much. As you may infer, I prefer the top version and it is also the version used by most libraries.

In [1]:
import numpy
import numpy.linalg

## Converting between state space and transfer function forms

There is good support in various libraries for converting systems with numeric coefficients between transfer function and state space representation.

### Scipy.signal

The `scipy.signal` library handles conversion between transfer function coefficients and state space matrices easily. Note that `scipi.signal` only handles SISO transfer functions.

$$G(s) = \frac{3s^2 + 7s + 15}{s^3 + 7s^2 + 14s + 8}$$

In [102]:
import scipy.signal

In [112]:
G = scipy.signal.lti([3,7,15], [1, 7,14,8])

In [113]:
G

TransferFunctionContinuous(
array([ 3.,  7., 15.]),
array([ 1.,  7., 14.,  8.]),
dt: None
)

This object allows us to access the numerator and denominator

In [114]:
G.num, G.den

(array([ 3.,  7., 15.]), array([ 1.,  7., 14.,  8.]))

To convert to state space, we can use the `.to_ss()` method

In [115]:
Gss = G.to_ss()

In [116]:
Gss.A, Gss.B, Gss.C, Gss.D

(array([[ -7., -14.,  -8.],
        [  1.,   0.,   0.],
        [  0.,   1.,   0.]]),
 array([[1.],
        [0.],
        [0.]]),
 array([[ 3.,  7., 15.]]),
 array([[0.]]))

We can build another object using the state space matrices instead of the Laplace form

In [117]:
G2ss = scipy.signal.lti(Gss.A, Gss.B, Gss.C, Gss.D)
G2ss

StateSpaceContinuous(
array([[ -7., -14.,  -8.],
       [  1.,   0.,   0.],
       [  0.,   1.,   0.]]),
array([[1.],
       [0.],
       [0.]]),
array([[ 3.,  7., 15.]]),
array([[0.]]),
dt: None
)

We can convert to transfer function form using `.to_tf()` (there is a small warning about bad coefficients, but the answer is reliable).

In [118]:
G2 = G2ss.to_tf()

C:\Users\reube\Anaconda3\lib\site-packages\scipy\signal\filter_design.py:1622: BadCoefficients: Badly conditioned filter coefficients (numerator): the results may be meaningless
  "results may be meaningless", BadCoefficients)


We can now access the numerator and denominator again:

In [119]:
G2.num, G2.den

(array([ 3.,  7., 15.]), array([ 1.,  7., 14.,  8.]))

Instead of building objects we can also use the functions in `scipy.signal.lti_conversion`:

In [120]:
scipy.signal.lti_conversion.tf2ss(1, [1, 1])

(array([[-1.]]), array([[1.]]), array([[1.]]), array([[0.]]))

In [121]:
scipy.signal.lti_conversion.ss2tf(-1, 1, 1, 0)

(array([[0, 1]]), array([1., 1.]))

### Control library

The control library (at least from version 0.8.0) does a good job with these conversions as well.

In [122]:
import control

In [123]:
Gtf = control.tf([3,7,15], [1, 7,14,8])
Gtf


   3 s^2 + 7 s + 15
----------------------
s^3 + 7 s^2 + 14 s + 8

In the control library we convert the system using `ss` (short for state space) to get a State Space representation:

In [124]:
Gss = control.ss(Gtf)
Gss

A = [[ -7. -14.  -8.]
 [  1.   0.   0.]
 [  0.   1.   0.]]

B = [[1.]
 [0.]
 [0.]]

C = [[ 3.  7. 15.]]

D = [[0.]]

In [111]:
Gss.A

matrix([[-1.]])

## Symbolic conversion

It is easy to convert state space models to transfer functions since the Laplace transform is a linear operator:

$$ \dot{x} = Ax + Bu \quad \therefore \quad sX(s) = AX(s) + BU(s) \quad X(s) = (sI - A)^{-1}BU(s)$$
$$ y = Cx + Du \quad \therefore \quad Y(s) = CX(s) + DU(s) \quad Y(s) = \underbrace{(C(sI - A)^{-1}B + D)}_{G(s)}U(s)$$

This conversion is handled for symbolic matrices by `tbcontrol.symbolic.ss2tf`

In [125]:
import sympy

In [126]:
import tbcontrol
tbcontrol.expectversion('0.1.8')
import tbcontrol.symbolic

In [127]:
s = sympy.symbols('s')

In [128]:
A, B, C, D = [sympy.Matrix(m) for m in [G2ss.A, G2ss.B, G2ss.C, G2ss.D]]

In [129]:
A, B, C, D

(Matrix([
 [-7.0, -14.0, -8.0],
 [ 1.0,   0.0,  0.0],
 [ 0.0,   1.0,  0.0]]),
 Matrix([
 [1.0],
 [0.0],
 [0.0]]),
 Matrix([[3.0, 7.0, 15.0]]),
 Matrix([[0.0]]))

In [130]:
G = tbcontrol.symbolic.ss2tf(A, B, C, D, s)
G

Matrix([[3.0*s**2/(-1.0*s*(-s*(s + 7.0) - 14.0) + 8.0) - 7.0*s/(1.0*s*(-s*(s + 7.0) - 14.0) - 8.0) + 15.0/(-s*(-s*(s + 7.0) - 14.0) + 8.0)]])

Note that `ss2tf` returns a sympy Matrix. To get the SISO result, we need to index into the matrix:

In [131]:
G[0,0].simplify()

1.0*(3.0*s**2 + 7.0*s + 15.0)/(1.0*s**3 + 7.0*s**2 + 14.0*s + 8.0)

## Analysis

Notice that the roots of the characteristic function correspond with the eigenvalues of the A matrix. The numerator and denominator of control transfer functions are stored as lists of lists to accomodate MIMO systems.

In [132]:
Gtf.pole()

array([-4., -2., -1.])

In [133]:
numpy.roots(Gtf.den[0][0])

array([-4., -2., -1.])

In [134]:
Eigen_values , Eigen_vector = numpy.linalg.eig(Gss.A)
Eigen_values

array([-4., -2., -1.])